## Volatility

The trully interesting exercises in this chapter are in the _Further Questions_ section. 10.20 and 10.22 are my vavourite ones. I created [dedicated classes for estimating EWMA and GARCH parameters](https://github.com/ilchen/RMaFI/blob/master/volatility/parameter_estimators.py) to tackle problems similar to those scratched in these exercises.

I don't want to repeate the exercises per see, so below you will find calculations inspired by them.

#### Estimating the &lambda; parameter for EWMA

In [1]:
import pandas_datareader.data as web
import datetime

from volatility import parameter_estimators

In [2]:
# Looking at an asset price changes between 7th July 2005 and 7th July 2010
start = datetime.datetime(2005, 7, 27)
end = datetime.datetime(2010, 7, 27)

# I use Yahoo Finance to fetch asset prices, in this case looking at the GBPUSD currency pair
data = web.get_data_yahoo('GBPUSD=X', start, end)
asset_prices_series = data['Adj Close']

In [3]:
# Inspired by exercise by Ex 10.20
ch10_ewma_min_diff = parameter_estimators.EWMAMinimumDifferenceParameterEstimator(asset_prices_series)
print('Optimal value for λ using the minimum difference method: %.5f' % ch10_ewma_min_diff.lamda)

Objective function: -0.00000 after 14 iterations
Optimal value for λ using the minimum difference method: 0.93750


In [4]:
ch10_ewma = parameter_estimators.EWMAParameterEstimator(asset_prices_series)
print('Optimal value for λ when using the EWMA: %.5f' % ch10_ewma.lamda)

Objective function: 11848.63466 after 15 iterations
Optimal value for λ when using the EWMA: 0.95849


#### Estimating the &omega;, &alpha;, and &beta; parameters for GARCH(1, 1)
This takes much longer than the EWMA estimator given that we need to solve a bounded and constrained optimization problem in three parameters as opposed to just a bounded optimization in one variable without any constraints for EWMA.

In [5]:
ch10_garch = parameter_estimators.GARCHParameterEstimator(asset_prices_series)
print('Optimal values for GARCH(1, 1) parameters:\n\tω=%.12f, α=%.5f, β=%.5f'
      % (ch10_garch.omega, ch10_garch.alpha, ch10_garch.beta))

/usr/local/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Objective function: 11857.54490 after 108 iterations
Optimal values for GARCH(1, 1) parameters:
	ω=0.000000293641, α=0.04649, β=0.94730
